### MACHINE LEARNING PROJECT I

#### TEAM MEMBERS: ASLI YORUSUN - ERDEM BOCUGOZ - SERIF SONER SERBEST

In [48]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from proj1_helpers import *
from implementations import *
from test import *
from data_cleaning import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
prediction,data,id_ = load_csv_data("train.csv", sub_sample=False)

In [50]:
data.shape

(250000, 30)

### Data Cleaning & Standardization

In [ ]:
#data = np.where(data==-999, np.nan, data)
#null_counts = data.isnull().sum()
#print(null_counts)

data = np.delete(data, [4,5,6,12,26,27,28], 1)
data.shape

In [51]:
data = clean_data(data)

In [52]:
x, mean_x, std_x = standardize(data)
y, tx = build_model_data(prediction,x)

y.shape, tx.shape

((250000,), (250000, 31))

In [ ]:
#TEST LEAST SQUARES AND RIDGE REGRESSION
def load_data():
    """load data."""
    data = np.loadtxt("dataEx3.csv", delimiter=",", skiprows=1, unpack=True)
    x = data[0]
    y = data[1]
    return x, y

# load dataset
x, y = load_data()
print("shape of x {}".format(x.shape))
print("shape of y {}".format(y.shape))

x.shape, y.shape

In [45]:
#TEST LOG REG

def de_standardize(x, mean_x, std_x):
    """Reverse the procedure of standardization."""
    x = x * std_x
    x = x + mean_x
    return x

def visualization(y, x, mean_x, std_x, w, save_name):
    """visualize the raw data as well as the classification result."""
    fig = plt.figure()
    # plot raw data
    x = de_standardize(x, mean_x, std_x)
    ax1 = fig.add_subplot(1, 2, 1)
    males = np.where(y == 1)
    females = np.where(y == 0)
    ax1.scatter(
        x[males, 0], x[males, 1],
        marker='.', color=[0.06, 0.06, 1], s=20)
    ax1.scatter(
        x[females, 0], x[females, 1],
        marker='*', color=[1, 0.06, 0.06], s=20)
    ax1.set_xlabel("Height")
    ax1.set_ylabel("Weight")
    ax1.grid()
    # plot raw data with decision boundary
    ax2 = fig.add_subplot(1, 2, 2)
    height = np.arange(
        np.min(x[:, 0]), np.max(x[:, 0]) + 0.01, step=0.01)
    weight = np.arange(
        np.min(x[:, 1]), np.max(x[:, 1]) + 1, step=1)
    hx, hy = np.meshgrid(height, weight)
    hxy = (np.c_[hx.reshape(-1), hy.reshape(-1)] - mean_x) / std_x
    x_temp = np.c_[np.ones((hxy.shape[0], 1)), hxy]
    prediction = x_temp.dot(w) > 0.5
    prediction = prediction.reshape((weight.shape[0], height.shape[0]))
    ax2.contourf(hx, hy, prediction, 1)
    ax2.scatter(
        x[males, 0], x[males, 1],
        marker='.', color=[0.06, 0.06, 1], s=20)
    ax2.scatter(
        x[females, 0], x[females, 1],
        marker='*', color=[1, 0.06, 0.06], s=20)
    ax2.set_xlabel("Height")
    ax2.set_ylabel("Weight")
    ax2.set_xlim([min(x[:, 0]), max(x[:, 0])])
    ax2.set_ylim([min(x[:, 1]), max(x[:, 1])])
    plt.tight_layout()
    plt.savefig(save_name)

def load_data(sub_sample=True, add_outlier=False):
    """Load data and convert it to the metric system."""
    path_dataset = "height_weight_genders.csv"
    data = np.genfromtxt(
        path_dataset, delimiter=",", skip_header=1, usecols=[1, 2])
    height = data[:, 0]
    weight = data[:, 1]
    gender = np.genfromtxt(
        path_dataset, delimiter=",", skip_header=1, usecols=[0],
        converters={0: lambda x: 0 if b"Male" in x else 1})
    # Convert to metric system
    height *= 0.025
    weight *= 0.454
    return height, weight, gender


def sample_data(y, x, seed, size_samples):
    """sample from dataset."""
    np.random.seed(seed)
    num_observations = y.shape[0]
    random_permuted_indices = np.random.permutation(num_observations)
    y = y[random_permuted_indices]
    x = x[random_permuted_indices]
    return y[:size_samples], x[:size_samples]

# load data.
height, weight, gender = load_data()

# build sampled x and y.
seed = 1
y = np.expand_dims(gender, axis=1)
X = np.c_[height.reshape(-1), weight.reshape(-1)]
y, X = sample_data(y, X, seed, size_samples=200)
x, mean_x, std_x = standardize(X)
tx = np.c_[np.ones((y.shape[0], 1)), x]

y.shape, tx.shape

((200, 1), (200, 3))

### Gradient Descent

In [5]:
# Define the parameters of the algorithm.
max_iters = 1000
gamma = 0.00001

# Initialization
#initial_w = np.zeros(tx.shape[1])

#weights, loss = least_squares_GD(y, tx, initial_w, max_iters, gamma)

### Stochastic Gradient Descent

In [6]:
# Define the parameters of the algorithm.
max_iters = 1000
gamma = 0.00001

# Initialization
#initial_w = np.zeros(tx.shape[1])

#weights, loss = least_squares_GD(y, tx, initial_w, max_iters, gamma)

### Least Squares

In [74]:
weights, loss = least_squares(y, tx)

print(loss)
print(weights)

rmse = compute_loss(y, tx, weights)
print("Least Squares, rmse={loss}".format(loss=rmse)) 

0.07241289396950215
[-4.64988198  0.06344158]
Least Squares, rmse=0.07241289396950215


### Ridge Regression

In [7]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-10, 0, 50)
    # split data
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)
    # form tx
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)

    # ridge regression with different lambda
    rmse_tr = []
    rmse_te = []
    for ind, lambda_ in enumerate(lambdas):
        # ridge regression
        weight, loss = ridge_regression(y_tr, tx_tr, lambda_)
        rmse_tr.append(np.sqrt(2 * compute_loss(y_tr, tx_tr, weight)))
        rmse_te.append(np.sqrt(2 * compute_loss(y_te, tx_te, weight)))

        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
    plot_train_test(rmse_tr, rmse_te, lambdas, degree)
    
seed = 1
degree = 10
split_ratio = 0.9
#ridge_regression_demo(x, y, degree, split_ratio, seed)

### Logistic Regression

In [47]:
x.shape, y.shape

((200, 2), (200, 1))

In [55]:
def logistic_regression_gradient_descent_demo(y, tx):
    y = np.expand_dims(y, axis=1)
    # init parameters
    max_iter = 1000 # 3000
    threshold = 1e-8
    gamma = 0.01 # 0.1
    losses = []
    
    w = np.zeros((tx.shape[1], 1))
    print(y.shape)
    print(tx.shape)


    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)     
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))       
            pred = sigmoid(tx.dot(w))
            print (np.any(pred == 0)) 
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    # visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_gradient_descent")
    print("loss={l}".format(l=compute_log_loss(y, tx, w)))
    return w

w = logistic_regression_gradient_descent_demo(y, tx)

(250000, 1)
(250000, 31)
Current iteration=0, loss=0.693147180559945
False
Current iteration=100, loss=0.6106230547030241
False
Current iteration=200, loss=0.5782196199392977
False
Current iteration=300, loss=0.5603983556493879
False
Current iteration=400, loss=0.5492383715980815
False
Current iteration=500, loss=0.5416759569568736
False
Current iteration=600, loss=0.5362579718214529
False
Current iteration=700, loss=0.5322084407445309
False
Current iteration=800, loss=0.5290768081249816
False
Current iteration=900, loss=0.5265847340360554
False
loss=0.5245520334491464


### Cross Validation

In [36]:
def cross_validation_demo(y,x):
    seed = 32
    degree = 10
    k_fold = 4
    step = 15
    lambdas = np.logspace(-10, 0, step)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    # ***************************************************
    # INSERT YOUR CODE HERE
    rmse_tr = []
    rmse_te = []
    progress = step * k_fold
    for ind, lambda_ in enumerate(lambdas):
        # ridge regression
        for k in range(4):
            print("Progress: " + str(progress) + " / " + str(step * k_fold))
            progress = progress - 1
            [a,b] = cross_validation(y,x,k_indices,k,lambda_,degree)
            rmse_tr.append(a)
            rmse_te.append(b)
    rmse_tr = np.asarray(rmse_tr).reshape(-1,4)
    rmse_te = np.asarray(rmse_te).reshape(-1,4)
    rmse_tr = np.mean(rmse_tr,axis=1)
    rmse_te = np.mean(rmse_te,axis=1)
    # cross validation: TODO
    # ***************************************************   
    cross_validation_visualization(lambdas,rmse_tr,rmse_te)
#cross_validation_demo(y,tx)

In [10]:
#print(rmse_te)

### TEST

In [7]:
test_label,test_data,test_id_ = load_csv_data("test.csv", sub_sample=False)

In [17]:
test_data = clean_data(test_data)

In [8]:
x_test,mean_test,std_x_test = standardize(test_data)
y_test,tx_test = build_model_data(test_label,x_test)
tx_test_poly = build_poly(tx_test, degree = 10)

In [11]:
#weight = logistic_regression_gradient_descent_demo(y,tx)
weights = logistic_regression_gradient_descent_demo(y, tx)

/Users/erdembocugoz/Desktop/Machine Learning/MLProject1/implementations.py:130: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))


loss=184883.9378858205


In [12]:
y_pred = predict_labels(weights,tx_test)

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [13]:
create_csv_submission(test_id_,y_pred,"logisticReg_submission.csv")